In [ ]:
import matplotlib.pyplot as plt
from math import floor, log10
import netCDF4 as nc
import numpy as np
import os
import pyvista as pv
import vtk

In [ ]:
def netcdf_copy(dsin, fnamein, datain):
    assert datain.ndim == 1, "oops! expected only 1D datain"
    
    # output file
    fnameout, ext = os.path.splitext(fnamein)
    fnameout = os.path.join(f"{fnameout}_synthetic{ext}")
    
    dsout = nc.Dataset(fnameout, "w", format=dsin.file_format)

    dnameout = None
    meshout = None
    coordinates = []
    
    # copy dimensions
    for dname, the_dim in dsin.dimensions.items():
        #print(dname, the_dim.size)
        if the_dim.size == datain.shape[0]:
            dnameout = dname
        dsout.createDimension(dname, the_dim.size if not the_dim.isunlimited() else None)

    # copy variables
    for v_name, varin in dsin.variables.items():
        outVar = dsout.createVariable(v_name, varin.datatype, varin.dimensions)
        #print(v_name, varin.datatype, varin.dimensions)
    
        # copy variable attributes
        attrs = {k: varin.getncattr(k) for k in varin.ncattrs()}
        outVar.setncatts(attrs)
        outVar[:] = varin[:]
        
        if "cf_role" in attrs and attrs["cf_role"] == "mesh_topology":
            meshout = v_name
        if len(varin.dimensions) == 1 and varin.dimensions[0] == dnameout:
            coordinates.append(v_name)
            
    # create the new data variable
    assert dnameout is not None, "oops! no output dimension name found"
    assert meshout is not None, "oops! no output mesh topology found"
    outVar = dsout.createVariable("synthetic", datain.dtype, (dnameout,))
    outVar[:] = datain
    coordinatesout = ' '.join(coordinates)
    attrs = dict(long_name="synthetic", units="1", mesh=meshout, coordinates=coordinatesout)
    outVar.setncatts(attrs)
    
    # copy global attributes
    dsout.setncatts({k: dsin.getncattr(k) for k in dsin.ncattrs()})
    
    # close the output file
    dsout.close()

In [ ]:
def to_unstructured(fname, node_face, node_x, node_y, radius=None, start_index=True, panel=None, synthetic=True):
    if radius is None:
        radius = 1.0
        
    ds = nc.Dataset(fname)
    node_face = ds.variables[node_face][:].data
    node_x = ds.variables[node_x][:].data
    node_y = ds.variables[node_y][:].data

    # account for start_index = 1
    if start_index:
        node_x = np.concatenate(([0], node_x))
        node_y = np.concatenate(([0], node_y))
    
    # convert lat/lon to cartesian coordinates
    node_face_x = node_x[node_face]
    node_face_y = 90.0 - node_y[node_face]
    
    node_face_x_rad = np.radians(node_face_x)
    node_face_y_rad = np.radians(node_face_y)
    
    x = radius * np.sin(node_face_y_rad) * np.cos(node_face_x_rad)
    y = radius * np.sin(node_face_y_rad) * np.sin(node_face_x_rad)
    z = radius * np.cos(node_face_y_rad)
    
    # set the VTK cell type and number of vertices
    vtk_cell_type = vtk.VTK_QUAD
    N_nodes = 4
    N_panels = 6
    
    # create unstructured grid
    points = np.vstack((np.ravel(x), np.ravel(y), np.ravel(z))).T
    if panel is not None:
        from collections import Iterable

        PN = points.shape[0] // N_panels
        if isinstance(panel, Iterable):
            panel_points = []
            for p in sorted(panel):
                panel_points.append(points[p*PN:(p+1)*PN])
            points = np.concatenate(tuple(panel_points))
        else:
            points = points[panel*PN:(panel+1)*PN]

    N_points = points.shape[0]
    N_faces = N_points // N_nodes
    N_faces_per_panel = N_faces // N_panels
    #print(N_points, N_faces, N_faces_per_panel, N_nodes)
    cell_type = np.broadcast_to(np.array([vtk.VTK_QUAD], np.uint8), (N_faces,))
    cells = np.ravel(np.hstack((np.broadcast_to(np.array([N_nodes], np.int8), (N_faces, 1)),
                                np.arange(0, N_points).reshape((-1, N_nodes)))))
    offset = np.arange(0, cells.shape[0], N_nodes + 1)
    
    ugrid = pv.UnstructuredGrid(offset, cells, cell_type, points)
    
    # add some synthetic data to the cells of each panel
    if synthetic:
        dtype = np.int32
        cell_array = np.ones(N_faces, dtype=dtype)
        panel_offset = 10**(floor(log10(N_faces_per_panel)) + 1)
        print(panel_offset)

        for p in range(N_panels):
            data = np.arange(N_faces_per_panel, dtype=dtype) #+ p * panel_offset
            cell_array[N_faces_per_panel*p:N_faces_per_panel*(p+1)] = data

        ugrid.cell_arrays["faces"] = cell_array
        netcdf_copy(ds, fname, cell_array)
        
    ds.close()
    
    return ugrid

In [ ]:
fname = "/home/bill/pyvista/data/test/synthetic/mesh_C4.nc"
node_face = "example_C4_face_nodes"
node_x = "example_C4_node_x"
node_y = "example_C4_node_y"

In [ ]:
fname = "/home/bill/pyvista/data/test/synthetic/mesh_C12.nc"
node_face = "dynamics_face_nodes"
node_x = "dynamics_node_x"
node_y = "dynamics_node_y"

In [ ]:
fname = "/home/bill/pyvista/data/test/synthetic/mesh_C24.nc"
node_face = "dynamics_face_nodes"
node_x = "dynamics_node_x"
node_y = "dynamics_node_y"

In [ ]:
fname = "/home/bill/pyvista/data/test/synthetic/mesh_C48.nc"
node_face = "dynamics_face_nodes"
node_x = "dynamics_node_x"
node_y = "dynamics_node_y"

In [ ]:
fname = "/home/bill/pyvista/data/test/synthetic/mesh_C96.nc"
node_face = "dynamics_face_nodes"
node_x = "dynamics_node_x"
node_y = "dynamics_node_y"

In [ ]:
fname = "/home/bill/pyvista/data/test/synthetic/mesh_C1048.nc"
node_face = "dynamics_face_nodes"
node_x = "dynamics_node_x"
node_y = "dynamics_node_y"

In [ ]:
fname = "/home/bill/pyvista/data/test/real/qrclim.sst.ugrid.nc"
node_face = "dynamics_face_nodes"
node_x = "dynamics_node_x"
node_y = "dynamics_node_y"
data = "surface_temperature"

In [ ]:
ugrid = to_unstructured(fname, node_face, node_x, node_y, synthetic=False)

In [ ]:
p = pv.PlotterITK()
p.add_mesh(ugrid)
#p.add_points(ugrid.points, color="green")
p.show()

In [ ]:
mesh_fname = "ugrid_C1048.vtk"
ugrid.save(mesh_fname)

In [ ]:
!ls -l *.vtk

In [ ]:
mesh = pv.read("ugrid_C1048.vtk")

In [ ]:
mesh

In [ ]:
p = pv.PlotterITK()
p.add_mesh(mesh)
#p.add_points(mesh.points, color="red")
p.show()